<a href="https://colab.research.google.com/github/rdukewiesenb/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [68]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 parse_dates=['SALE DATE'],
                 index_col='SALE DATE')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [69]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [70]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

1. Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [71]:
# Make a new dataset that only inlcudes any observations that include
# '01 ONE FAMILY DWELLINGS' in 'BUILDING_CLASS_CATEGORY'
# and $100,000 < 'SALE_PRICE' < $2,000,000

one_fam = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & 
                 (df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000)]
# ['BUILDING_CLASS_CATEGORY' == '01 ONE FAMILY DWELLINGS', 'SALE_PRICE' == range(100000, 2000000)]

# # Building class category
# fam_dwell = df['BUILDING_CLASS_CATEGORY', 'SALE_PRICE'].isin(filter)
# one_fam = df[fam_dwell]
# print(one_fam.shape)

# # Sale price
# mask = (one_fam['SALE_PRICE'] >= 100000 & )

print("one_fam shape: ", one_fam.shape)
one_fam.dtypes

one_fam shape:  (3164, 20)


BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
dtype: object

In [72]:
one_fam.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000


In [32]:
# Move 'SALE_DATE' to index
# We set this column as the index back up in the cell
# where we imported the CSV

In [74]:
# I know that at least one of my features has to be categorical,
# so I'm going to extract the years in the 'Year_BUILT' column,
# and categorize them by decade

# What are the min and max years built?
print('Min: ', one_fam['YEAR_BUILT'].min())
print('Max: ', one_fam['YEAR_BUILT'].max())

Min:  1890.0
Max:  2018.0


In [91]:
# Let's make a new column for the decade built

bins = [1890, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]
labels = ['1890', '1900', '1910', '1920', '1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010']

one_fam['DECADE_BUILT'] = pd.cut(one_fam['YEAR_BUILT'], bins=bins, labels=labels, right=False)
print('Column type: ', one_fam['DECADE_BUILT'].dtypes)
one_fam.head()

Column type:  category


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,DECADE_BUILT
SALE DATE,,,,,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,1930
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,1940
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,1920
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,1890
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,1940


2. Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [ ]:
# I want to convert all zip codes to ints, and then to strings
# so that the zip codes no longer have decimal points
# but I couldn't figure it, considering my circumstances


# one_fam['ZIP_CODE'] = int(one_fam['ZIP_CODE'])
# one_fam['ZIP_CODE'] = one_fam['ZIP_CODE'].astype(str)

In [123]:
# Steps to completing a train/test split

# 1. establish target and features
# the target is what we're predicting
# let's filter for only the features I want:
  # 'DECADE_BUILT'
  # 'BOROUGH'

target = 'SALE_PRICE'
y = one_fam[target]
X = one_fam.drop([target]+['EASE-MENT', 'NEIGHBORHOOD', 'ADDRESS', 'APARTMENT_NUMBER', 'BUILDING_CLASS_CATEGORY'], axis=1)

In [124]:
# double check my work
print('X shape: ', X.shape)
X.head()

X shape:  (3164, 15)


,BOROUGH,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,DECADE_BUILT
SALE DATE,,,,,,,,,,,,,,,
2019-01-01,3,1,5495,801,A9,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,1930
2019-01-01,4,1,7918,72,A1,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,1940
2019-01-02,2,1,4210,19,A1,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,1920
2019-01-02,3,1,5212,69,A1,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,1890
2019-01-02,3,1,7930,121,A5,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,1940


In [125]:
# 2. Split dataset into training and validation

# our cutoff is April 2019
cutoff = '2019-04-01'
mask = X.index < cutoff

# Training
X_train, y_train = X.loc[mask], y.loc[mask]

# Validation
X_val, y_val = X.loc[~mask], y.loc[~mask]

In [126]:
# Use an assert statement to check for any errors

assert X_train.shape[0] + X_val.shape[0] == X.shape[0]

In [127]:
# 3. Set the baseline

# our default answer will be the y_train mean
print('y_train mean: ', y_train.mean())

y_train mean:  621011.2721493841


In [128]:
from sklearn.metrics import mean_absolute_error

# our baseline
print('Baseline MAE: ', mean_absolute_error(y_train, [y_train.mean()]*len(y_train)))

Baseline MAE:  216497.12357684076


In [131]:
# 4. Build our model

from sklearn.preprocessing import OneHotEncoder

# Because we are building a ridge regression model,
# before we can build our model,
# we must first transform it

# Steps to transforming our data:

# 1. Instantiate the transformer
transformer1 = OneHotEncoder

# 2. Fit transformer to data
transformer1.fit(X_train, y_train)

# 3. Transform the training data
XT_train = transformer1.transform(X_train)

AttributeError: ignored

In [122]:
# 5. SelectKBest

from sklearn.feature_selection import SelectKBest

# Steps to do a SelectKBest:

# 1. Instantiate the transformer
transformer2 = SelectKBest(k=5)

# 2. Fit model to the training data
transformer2.fit(XT_train, y_train)

# 3. Transform the training data
XTT_train = transformer2.transform(XT_train)

ValueError: ignored

In [ ]:
# Using predictions to build the model

# Steps to creating predictions for our training metrics:
from sklearn.linear_model import LinearRegression

# 1. Instantiate model
predictor = LinearRegression()

# 2. Fit model to the data
predictor.fit(X_train, y_train)

# 3. Make predictions
y_pred = predictor.predit(X_train)

# 4. Establish training MAE
print('Training MAE: ', mean_absolute_error(y_train, y_pred))

# 5. Establish validation metrics
XT_val = transformer.transform(X_val)

# 6. Establish validation MAE
print('Validation MAE: ', mean_absolute_error(y_val, y_pred))

In [ ]:
# 6. Ridge Regression

from sklearn.linear_model import Ridge

# Steps to create rdige model:

# 1. Instantiate the model
predictor2 = Ridge()

# 2. Fit model to training data
predictor2.fit(XTT_train, y_train)

In [ ]:
# 7. Visualizing Coefficients

# Steps to visualizing coefficients:

# 1. Transform the ridge data
transformer = OneHotEncoder
XT_train = transformer.fit(X_train)